In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Load the training data
train_data = pd.read_csv("data/train_transformed.csv")

In [3]:
eva_data = pd.read_csv("data/evaluation_transformed.csv")

In [4]:
train_data.head()

,id,user_verified,user_statuses_count,user_followers_count,user_friends_count,ratio_friends_followers,mention_exist,mention_count,url_exist,url_count,...,tf_idf_0,tf_idf_1,tf_idf_2,tf_idf_3,tf_idf_4,tf_idf_5,tf_idf_6,tf_idf_7,tf_idf_8,tf_idf_9
0,0,0,68460,1101,1226,1.112523,0,0,0,0,...,-1.606137e-15,-1.992248e-13,-6.077335e-13,-1.726278e-12,7.402787e-13,2.375980e-14,2.692723e-13,5.657295e-13,-2.124336e-14,9.444088e-14
1,1,0,309,51,202,3.884615,0,0,0,0,...,1.206801e-02,2.715082e-03,1.318767e-02,3.157979e-02,2.662453e-02,1.621955e-02,-2.204838e-02,9.316040e-03,3.288260e-03,7.966161e-02
2,2,0,3241,1675,2325,1.387232,0,0,0,0,...,5.095493e-02,-1.131896e-03,3.852378e-02,1.163440e-01,1.542300e-01,2.938599e-01,4.018045e-01,1.909630e-01,3.218782e-01,-4.933187e-02
3,3,0,32327,667,304,0.455090,0,0,0,0,...,8.873053e-03,1.403727e-03,3.399734e-03,1.401861e-02,9.761796e-03,9.215406e-03,6.693422e-03,8.483338e-03,-2.502589e-03,1.261141e-02
4,4,0,581,42,127,2.953488,0,0,0,0,...,2.774364e-02,-1.756079e-03,8.245230e-03,3.155745e-02,1.385880e-02,5.119571e-03,3.617241e-03,1.908667e-02,-1.212654e-02,1.883319e-02


In [5]:
# scsplit method is used in order to split our regression data in a stratisfied way and keep a similar distribution of retweet counts between the two sets
X_train_all, X_test_all, y_train, y_test = scsplit(train_data, train_data['retweet_count'], stratify=train_data['retweet_count'], train_size=0.7, test_size=0.3)


In [6]:
features_need_scaled=['user_statuses_count', 'user_followers_count', 'user_friends_count', 'ratio_friends_followers', 'mention_count','url_count', 'hashtag_count', 'text_length']
scaler = preprocessing.StandardScaler()
scaler.fit(X_train_all[features_need_scaled])
X_train_all[features_need_scaled] = scaler.transform(X_train_all[features_need_scaled])
X_test_all[features_need_scaled] = scaler.transform(X_test_all[features_need_scaled])
eva_data[features_need_scaled] = scaler.transform(eva_data[features_need_scaled])
X_train_all.head()

,id,user_verified,user_statuses_count,user_followers_count,user_friends_count,ratio_friends_followers,mention_exist,mention_count,url_exist,url_count,...,tf_idf_0,tf_idf_1,tf_idf_2,tf_idf_3,tf_idf_4,tf_idf_5,tf_idf_6,tf_idf_7,tf_idf_8,tf_idf_9
312340,312340,0,-0.418202,-0.095698,-0.116456,0.110003,0,-0.231206,0,-0.674199,...,0.026978,0.004563,0.026511,0.089760,0.116198,0.478829,-0.309716,-0.053400,-0.033924,-0.033351
136930,136930,0,-0.417182,-0.095786,-0.155118,0.194613,0,-0.231206,0,-0.674199,...,0.195770,-0.153774,0.324744,0.118359,-0.057019,-0.023696,0.004907,0.016485,-0.008501,0.034761
299435,299435,0,-0.283807,-0.095260,-0.138928,-0.202147,0,-0.231206,1,1.367238,...,0.218284,-0.032150,-0.070487,-0.054131,0.049342,0.024478,0.001115,0.024649,-0.007772,0.048858
597563,597563,0,-0.307979,-0.095594,-0.112720,-0.056442,0,-0.231206,0,-0.674199,...,0.020886,0.003246,-0.003002,0.034441,0.020268,0.009500,0.005160,0.020818,-0.003763,0.048333
622933,622933,0,-0.326418,-0.094967,0.022230,-0.060231,0,-0.231206,1,1.367238,...,0.044784,0.010294,0.002660,0.075867,0.010516,0.027049,0.024499,0.201043,-0.155097,-0.039574


## Support vector machine - Random Forest

In [9]:
def svr(features_selected):
    X_train = X_train_all[features_selected]
    X_test = X_test_all[features_selected]
    svr = LinearSVR()
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))
    return svr

In [18]:
def svr_rf(X_train, y_train, max_depth, n_estimators):
    # svr
    svr = LinearSVR()
    y_train_log = np.log(y_train+1)
    svr.fit(X_train, y_train_log) 
    svr_y_train_predict = svr.predict(X_train) # this produces a ndarray
    
    # random forest
    # for training residual RF
    rf_y_train = y_train_log - svr_y_train_predict
    
    reg = RandomForestRegressor(max_depth = max_depth,   
                                n_estimators = n_estimators, 
                                random_state = 7,  
                                n_jobs = 10, 
                                verbose = 5)  

    reg.fit(X_train, rf_y_train)
    return svr, reg

### Cross Validation

In [13]:
X_train = train_data[features_selected]
y_train = train_data['retweet_count']
eval_data = pd.read_csv("data/evaluation_transformed.csv")
X_val = eval_data[features_selected]


features_need_scaled=['user_statuses_count', 'user_followers_count', 'user_friends_count', 'ratio_friends_followers', 'mention_count','url_count', 'hashtag_count', 'text_length']
scaler = preprocessing.StandardScaler()
scaler.fit(X_train[features_need_scaled])
X_train[features_need_scaled] = scaler.transform(X_train[features_need_scaled])
X_val[features_need_scaled] = scaler.transform(X_val[features_need_scaled])

In [14]:
X_train.shape

(665777, 19)

In [15]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 43)
KFold(5, shuffle = True, random_state = 43).split(X_train, y_train)

<generator object _BaseKFold.split at 0x00000271A57140C8>

In [16]:
features_selected=['user_verified', 'user_statuses_count', 'user_followers_count',
                 'user_friends_count', 'ratio_friends_followers', 'mention_exist',
                 'mention_count', 'url_exist', 'url_count', 'hashtag_exist',
                 'hashtag_count', 'weekend', 'text_length', 'sentiment_pos', 
                 'sentiment_neg', 'sentiment_neu', 'tf_idf_0', 'tf_idf_1', 'tf_idf_2'] 

In [19]:
results_cv=[]

for train_index, test_index in kf.split(np.array(X_train)):
    X_train_tmp, X_test_tmp = pd.DataFrame(np.array(X_train)[train_index]), pd.DataFrame(np.array(X_train)[test_index])
    y_train_tmp, y_test_tmp = y_train[train_index], y_train[test_index]
    
    # normalization
     
    features_need_scaled=['user_statuses_count', 'user_followers_count', 'user_friends_count', 'ratio_friends_followers', 'mention_count','url_count', 'hashtag_count', 'text_length']
    X_train_tmp.columns = features_selected
    X_test_tmp.columns = features_selected
    scaler = preprocessing.StandardScaler()
    scaler.fit(X_train_tmp[features_need_scaled])
    X_train_tmp[features_need_scaled] = scaler.transform(X_train_tmp[features_need_scaled])
    X_test_tmp[features_need_scaled] = scaler.transform(X_test_tmp[features_need_scaled])
    
    # train
    svr, reg = svr_rf(X_train_tmp, y_train_tmp, 18, 100)
    
    # predict
    svr_predict_y_log = svr.predict(X_test_tmp)

    rf_predict_y_log = reg.predict(X_test_tmp)

    y_predict = np.exp(svr_predict_y_log + rf_predict_y_log) -1

    print(mean_absolute_error(y_test_tmp, y_predict))
    results_cv.append(mean_absolute_error(y_test_tmp, y_predict))

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100building tree 4 of 100

building tree 5 of 100building tree 6 of 100
building tree 7 of 100
building tree 8 of 100

building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100building tree 36 of 100

building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   55.7s


building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100building tree 89 of 100

building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.3s finished


...

[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:  1.5min


building tree 64 of 100building tree 65 of 100

building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:  2.6min finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.3s finished


129.31193536939008


AttributeError: 'list' object has no attribute 'mean'

In [20]:
print(np.array(results_cv).mean())

137.87980415744846


## Evaluation and dump

In [17]:
X_train = train_data[features_selected]
y_train = train_data['retweet_count']
eval_data = pd.read_csv("data/evaluation_transformed.csv")
X_val = eval_data[features_selected]


features_need_scaled=['user_statuses_count', 'user_followers_count', 'user_friends_count', 'ratio_friends_followers', 'mention_count','url_count', 'hashtag_count', 'text_length']
scaler = preprocessing.StandardScaler()
scaler.fit(X_train[features_need_scaled])
X_train[features_need_scaled] = scaler.transform(X_train[features_need_scaled])
X_val[features_need_scaled] = scaler.transform(X_val[features_need_scaled])

In [20]:
y_train.shape

(665777,)

In [21]:
svr2, reg2 = svr_rf(X_train, y_train, 18, 100)

Prediction error: 374.5686731510398
building tree 1 of 100building tree 2 of 100building tree 3 of 100



[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.



building tree 4 of 100
building tree 5 of 100
building tree 6 of 100building tree 7 of 100
building tree 8 of 100

building tree 9 of 100building tree 10 of 100

building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 1

[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:  1.5min


building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:  2.5min finished


In [22]:
svr_predict_y_log_eva = svr1.predict(X_val)

rf_predict_y_log_eva = reg1.predict(X_val)

y_predict_eva = np.exp(svr_predict_y_log_eva + rf_predict_y_log_eva) - 1

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    0.4s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.7s finished


In [23]:
# Dump the results into a file that follows the required Kaggle template
with open("prediction/svr_rf_predictions.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "NoRetweets"])
    for index, prediction in enumerate(y_predict_eva):
        writer.writerow([str(eval_data['id'].iloc[index]) , str(int(prediction))])